## Environment Set-Up
Setting up a virtual environment is a crucial step in managing your Python project's dependencies effectively. 

###  1. Create a Virtual Environment
Please use Python 3.11 version for the workshop.

Run the following command to create a virtual environment in your project directory. 

Replace `.venv` with your desired name for the virtual environment folder if needed:

`python -m venv .venv`


###  2. Activate the Virtual Environment
Activate the virtual environment using the appropriate command for your operating system:
- On Linux/macOS:

    `source .venv/bin/activate`
    
- On Windows (Command Prompt)

    `.venv\Scripts\activate
`

- On Windows (PowerShell):

    `.venv\Scripts\Activate.ps1`

###  3. Upgrade `pip`
After activating the virtual environment, upgrade `pip` to ensure you have the latest version:

`pip install --upgrade pip`

###  4. Install Project Dependencies
Use `pip` to install all the required dependencies for the project as specified in the `requirements.txt` file:

`pip install -r requirements.txt`

###  5. Install Azure CLI SDK
To install the Azure CLI SDK, please visit: https://learn.microsoft.com/en-us/cli/azure/install-azure-cli

### 6. Log in to Azure
Run the following command to log in to Azure:
`az login`

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get tenant ID and subscription ID from environment variables
tenant_id = os.getenv("AZURE_TENANT_ID")
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")

if not tenant_id or not subscription_id:
    raise ValueError("AZURE_TENANT_ID or AZURE_SUBSCRIPTION_ID is not set in the .env file.")

# Use the tenant ID and subscription ID to log in and set the subscription context
! az login --tenant "{tenant_id}"
! az account set --subscription "{subscription_id}"

## Create a project in Azure AI Foundry

1. Follow the guidelines in this article (https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-projects?tabs=ai-studio). If you don't have them already, make sure to create new AI services and AI Search resources.

   <Image src='docs/create_project_in_azure_ai_foundry.png'>


2. **Obtain the Connection String**  
   Copy the connection string from your AI Studio project. It should follow this format: `eastus.api.azureml.ms;12345678-abcd-1234-9fc6-62780b3d3e05;my-resource-group;my-project-name`
   - Ref: You can find the connection string in the Azure AI Foundry project settings
     - https://learn.microsoft.com/en-us/azure/ai-foundry/media/how-to/projects/project-settings.png#lightbox

3. **Deploy gpt-4o-mini model**  
   In the `Models + Endpoints` section, deploy a gpt-4o-mini model with the deoloyment name as "gpt-4o-mini".
   - Note: When trying to create an Enterprise Agent in existing AI Project, if you get "Enterprise agents is only supported when project has 1 Azure Open AI connection"
   - Please refer to the following issue for more details:
     - https://github.com/Azure/azure-sdk-for-python/issues/38921

4. **Set Environment Variables**  
Ensure you set the required environment variables in `.env` file.

## (Optional) Create a TimeGEN endpoint

1. **Find TimeGEN model in Model Catalog**

   <Image src='docs/time_gen_1.png'>


2. **Create an endpoint**  
   
   <Image src='docs/time_gen_2.png'>

3. **Set Environment Variables**  
Save endpoint and key variables in `.env` file.

In [1]:
# load environment variables from the .env file
import os
from dotenv import load_dotenv

load_dotenv()
print(os.getenv('CHAT_MODEL'))

gpt-4o-mini


In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv()

# Print the connection string
# print("Connection String:", os.environ.get("AIPROJECT_CONNECTION_STRING"))

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

connections = project_client.connections.list()
for connection in connections:
    print(connection)

In [ ]:
from azure.ai.projects.models import ConnectionType

connections = project_client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI,
)
for connection in connections:
    print(connection)

In [3]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01")

response = aoai_client.chat.completions.create(
    model=os.environ["CHAT_MODEL"],  # Model deployment name
    messages=[
        {
            "role": "user",
            "content": "How many feet are in a mile?",
        },
    ],
)

print(response.choices[0].message.content)

There are 5,280 feet in a mile.


# List all agents

In [ ]:
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    print(_agent)

# Delete all agents

In [ ]:
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    project_client.agents.delete_agent(_agent.id)
    print(f'agent {_agent.name} deleted')

# Create Agent

In [ ]:
##################################
# Original version, commented out
##################################
# from azure.ai.projects.models import CodeInterpreterTool


# # Create an instance of the CodeInterpreterTool
# code_interpreter = CodeInterpreterTool()


# # The CodeInterpreterTool needs to be included in creation of the agent
# agent = project_client.agents.create_agent(
#     model=os.environ["CHAT_MODEL"],
#     name="code_agent",
#     instructions="You are helpful agent",
#     tools=code_interpreter.definitions,
#     tool_resources=code_interpreter.resources,
# )

# print(f"Created agent, agent ID: {agent.id}")

In [ ]:
"""
This script checks if an agent named "code_agent" already exists.
If it exists, it uses the existing agent. Otherwise, it creates a new one.
The CodeInterpreterTool is used to define tools and resources for the agent.
"""

from azure.ai.projects.models import CodeInterpreterTool

# Create an instance of the CodeInterpreterTool
code_interpreter = CodeInterpreterTool()

# Check if the agent already exists
agents = project_client.agents.list_agents()

# Parse the agents list correctly
existing_agent = next(
    (agent for agent in agents.data if agent.name == "code_agent"),
    None
)

if existing_agent:
    # Use the existing agent
    agent_id = existing_agent.id
    print(f"Using existing agent, agent ID: {agent_id}")
else:
    # Create a new agent if it doesn't exist
    agent = project_client.agents.create_agent(
        model=os.environ["CHAT_MODEL"],
        name="code_agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    agent_id = agent.id
    print(f"Created new agent, agent ID: {agent_id}")

In [ ]:
# List all agents
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    print(_agent)

In [ ]:
# Create a thread to hold the conversation
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

In [ ]:
# Create a message
user_message = "Hello"
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, message ID: {message.id}, content: {message.content}")

In [ ]:
# Run the agent
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

In [46]:
# Display all messages in the thread
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

# Conversation
___
### **User** (2025-04-02 10:12:38 台北標準時間)
Hello
___
### **Assistant** (2025-04-02 10:12:41 台北標準時間)
Hello! How can I assist you today?
___

In [47]:
print(messages)

{'object': 'list', 'data': [{'id': 'msg_bS97i56rF9thOwn8Xq1eY3Bj', 'object': 'thread.message', 'created_at': 1743559961, 'assistant_id': 'asst_70EilY9SGkIdz3hbbAqw7PnH', 'thread_id': 'thread_6Uphwat9kyg1JOhJ7540x6xX', 'run_id': 'run_eeuxMeHhH1f8eiJFhsSLQD1B', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Hello! How can I assist you today?', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_EkVVa7kSceH9vyx6mWWyiNEX', 'object': 'thread.message', 'created_at': 1743559958, 'assistant_id': None, 'thread_id': 'thread_6Uphwat9kyg1JOhJ7540x6xX', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_bS97i56rF9thOwn8Xq1eY3Bj', 'last_id': 'msg_EkVVa7kSceH9vyx6mWWyiNEX', 'has_more': False}


In [48]:
user_message = "Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million"

In [ ]:
from pathlib import Path


thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, message ID: {message.id}")

# Run the agent
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")


messages = project_client.agents.list_messages(thread_id=thread.id)
display(Markdown(helper.get_conversation_md(messages)))

In [ ]:
from IPython.display import Image, display


# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image File ID: {image_content.image_file.file_id}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(
        file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {Path.cwd() / file_name}")

# Print the file path(s) from the messages
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths:")
    print(f"Type: {file_path_annotation.type}")
    print(f"Text: {file_path_annotation.text}")
    print(f"File ID: {file_path_annotation.file_path.file_id}")
    print(f"Start Index: {file_path_annotation.start_index}")
    print(f"End Index: {file_path_annotation.end_index}")
    
    saved_file_name = Path(file_path_annotation.text).name
    project_client.agents.save_file(
        file_id=file_path_annotation.file_path.file_id, file_name=saved_file_name)
    display(Image(filename=saved_file_name))
